In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ahkhalwai55","key":"08548f20f462e42e9f053fb77ae77384"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# change permission
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d tongpython/cat-and-dog

 95% 205M/217M [00:02<00:00, 45.6MB/s]
100% 217M/217M [00:02<00:00, 85.0MB/s]


In [16]:
pip install zipfile

  Could not find a version that satisfies the requirement zipfile (from versions: )
No matching distribution found for zipfile


In [18]:
from zipfile import ZipFile
file_name = "cat-and-dog.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [19]:
from zipfile import ZipFile
file_name = "training_set.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [20]:
from zipfile import ZipFile
file_name = "test_set.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
# Part 1 - Building the CNN

#import libraries
# we are not importing libraries used for csv files, as keras deals with all of these

from keras.models import Sequential
from keras.layers import Convolution2D #images are two dimensional. Videos are three dimenstional with time.
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


In [23]:
#initialize the classifier CNN
classifier = Sequential()

#applying convolution operation --> build the convolutional layer
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))
#32, 3, 3 --> 32 filters with 3 x 3 for each filter. 
#start with 32 filters, and then create more layers with 64, 128, 256, etc
#expected format of the images.
# 256, 256, 3 --> 3 color channels (RGB), 256 x 256 pixels. But when using CPU, 3, 64, 64 --> due to computational limitation

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  after removing the cwd from sys.path.


In [24]:
#Max Pooling --> create a pooling layer
classifier.add(MaxPooling2D(pool_size = (2,2)))
# 2 x 2 size --> commonly used to keep much information.

#Flattening --> creating a long vector.
classifier.add(Flatten()) #no parameters needed.

#classic ANN - full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
#common practice: number of hidden nodes between the number of input nodes and output nodes, and choose powers of 2
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
classifier.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

In [26]:
#Data augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('training_set', 
                                                    target_size = (64, 64), 
                                                    batch_size = 32,
                                                   class_mode = 'binary')
test_set = test_datagen.flow_from_directory('test_set',
                                                target_size = (64, 64),
                                                 batch_size = 32, 
                                                 class_mode = 'binary')

classifier.fit_generator(training_set, 
                         samples_per_epoch = 8005, 
                        nb_epoch = 2, 
                        validation_data = test_set, 
                        nb_val_samples = 2025)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=250, epochs=2, validation_steps=2025)`


Epoch 1/2
250/250 [==============================] - 181s 722ms/step - loss: 0.6677 - acc: 0.6181 - val_loss: 0.6605 - val_acc: 0.6303
Epoch 2/2
250/250 [==============================] - 176s 706ms/step - loss: 0.5875 - acc: 0.6947 - val_loss: 0.5571 - val_acc: 0.7230


4. Improving the model
There are two possible ways of reducing variane, which is making the model fit more to the train set.

Add more convolutional layers
This will allow more features to be detected prior to fitting to ANN. Make sure to not include input_dim, and include MaxPooling step. Flattening should be at the end of all the layers.
Add more fully-connected layer (hidden layers)
Catches more complex behaviors
Thank you for reading this kernel. If you found this helpful, please upvote the kernel or put a short comment below.